In [ ]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
!python -m spacy download en_core_web_lg
import spacy
from tqdm import tqdm
# from tqdm.notebook import tqdm_notebook

2023-11-04 23:07:09.501234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 23:07:09.501323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 23:07:09.501364: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 23:07:09.516011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 23:07:12.954835: W tensorflow/compiler/

In [ ]:
import warnings

# Use the "filterwarnings" function to suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/'SJSU Classes'/'*2023 Fall DATA298A'/'BERT data'
!ls

/content/drive/My Drive/SJSU Classes/*2023 Fall DATA298A/BERT data
298_compiled_travel_keywords.csv  processed_data.csv	    train_3.jsonl     travel_words.csv
bertopic_model.pkl		  processed_with_topic.csv  train_4.jsonl     worldcities.csv
bertopic_test.pkl		  train_0.jsonl		    train_5.jsonl
CallReddit.ipynb		  train_1.jsonl		    travel_df.csv
databricks-dolly-15k_travel.pkl   train_2.jsonl		    travel_regex.txt


In [ ]:
# Define the folder containing the JSONL files
cwd = os.getcwd()

# Get Data

In [ ]:
cities = pd.read_csv('worldcities.csv')['city'].to_list()
travel_words = pd.read_csv('travel_words.csv', header=None)[0].to_list()
cities_set = set(cities)
travel_words_set = set(travel_words)

print(len(cities))
print(len(cities_set))
print(len(travel_words))
print(len(travel_words_set))

44691
41343
315
134


In [ ]:
check_for_words = '''airbnb tent camping staycation explore temple tourism bites attraction sites bus shoestring
wanderlust backpacking hiking van road trip vagabond travel remote rural scenic landscape village map nomad flight
couchsurf cruise journey'''

In [ ]:
check_for_words = check_for_words.split()
check_for_words = set(check_for_words)
check_for_words

{'airbnb',
 'attraction',
 'backpacking',
 'bites',
 'bus',
 'camping',
 'couchsurf',
 'cruise',
 'explore',
 'flight',
 'hiking',
 'journey',
 'landscape',
 'map',
 'nomad',
 'remote',
 'road',
 'rural',
 'scenic',
 'shoestring',
 'sites',
 'staycation',
 'temple',
 'tent',
 'tourism',
 'travel',
 'trip',
 'vagabond',
 'van',
 'village',
 'wanderlust'}

# Manually Check and Add Words

In [ ]:
difference = check_for_words - travel_words_set
difference

{'airbnb',
 'attraction',
 'backpacking',
 'bites',
 'couchsurf',
 'explore',
 'landscape',
 'map',
 'remote',
 'rural',
 'shoestring',
 'sites',
 'staycation',
 'temple',
 'vagabond',
 'van',
 'village',
 'wanderlust'}

In [ ]:
len(difference)

18

In [ ]:
travel_words_set = travel_words_set.union(check_for_words)

In [ ]:
len(travel_words_set)

152

# Similar Word Vectors

In [ ]:
similarity_threshold = 0.3

In [ ]:
# Load spaCy with the medium English model (which includes word vectors)
nlp = spacy.load("en_core_web_lg")

# Dictionary to store unique similar words and their highest similarity score
unique_similar_words = {}

seed_words = travel_words_set

with tqdm(total=len(seed_words), unit="row", desc="Processing", dynamic_ncols=True) as pbar: #pbar.update(1) ###

  # Find similar words
  for word in seed_words:
      # Get the token for the seed word
      seed_token = nlp(word)

      # Find similar words using spaCy's word vectors
      similar_tokens = [token for token in nlp.vocab if not token.is_stop and token.is_alpha and
                        token.text.lower() != word and seed_token.similarity(token) > similarity_threshold]

      # Update the dictionary with unique similar words and their highest similarity score
      for token in similar_tokens:
          token_text = token.text.lower()
          similarity = seed_token.similarity(token)
          if token_text not in unique_similar_words or similarity > unique_similar_words[token_text]:
              unique_similar_words[token_text] = similarity

      pbar.update(1) ###

# Update task with timing information
task = f'finding similar word embeddings for {df.shape[0]} travel keywords'
print(f"\nElapsed time for {task}: {pbar.format_interval(pbar.last_print_t - pbar.start_t)}")

# Sort the unique similar words by their highest similarity score
sorted_similar_words = sorted(unique_similar_words.items(), key=lambda x: x[1], reverse=True)

Processing: 100%|██████████| 152/152 [00:02<00:00, 74.86row/s]


Elapsed time for finding similar word embeddings for 330 travel keywords: 00:01


In [ ]:
len(sorted_similar_words)

159

In [ ]:
# Print the top N similar words with their highest similarity scores
for word, score in sorted_similar_words[:10]:
    print(f"{word}: {score}")

park: 0.9425949131853893
tourist: 0.9073365769160389
car: 0.9062630961539234
sunset: 0.9046526279910705
staycation: 0.8668222753211425
baggage: 0.8622013385733944
map: 0.8615537332241894
jet: 0.8438389933465902
road: 0.8273038487152198
ski: 0.8223414724963717


In [ ]:
# Print the top N similar words with their highest similarity scores
for word, score in sorted_similar_words[-10:]:
    print(f"{word}: {score}")

globetrotter: 0.36223951288819944
calif: 0.36085339021724194
escapade: 0.353956946465806
ticket: 0.3487050972892803
mass: 0.3293919704648471
lovin: 0.3264586177555681
corp: 0.31782919824646644
colo: 0.3172704884769808
culture: 0.31120919383067125
jan: 0.30039504960907565


In [ ]:
similar_words = [t[0] for t in sorted_similar_words]
similar_words

['park',
 'tourist',
 'car',
 'sunset',
 'staycation',
 'baggage',
 'map',
 'jet',
 'road',
 'ski',
 'trip',
 'backpack',
 'camping',
 'zip',
 'fishing',
 'backpacking',
 'adventurous',
 'wanderer',
 'diving',
 'attraction',
 'navigation',
 'lag',
 'vacation',
 'reservation',
 'travel',
 'destination',
 'line',
 'cancellation',
 'touristy',
 'taxi',
 'boating',
 'exploration',
 'wandering',
 'discovery',
 'arrival',
 'visa',
 'sailing',
 'arrive',
 'resort',
 'expedition',
 'swimming',
 'snorkeling',
 'waterfall',
 'wanderlust',
 'amusement',
 'train',
 'route',
 'hiking',
 'rental',
 'airline',
 'scenic',
 'aircraft',
 'trail',
 'inclusive',
 'detour',
 'outdoor',
 'foreign',
 'excursion',
 'surfing',
 'airport',
 'sightseeing',
 'boarding',
 'adventure',
 'frontier',
 'ferry',
 'bicycle',
 'tent',
 'hotel',
 'driving',
 'lodge',
 'pilgrimage',
 'getaway',
 'explore',
 'backpacker',
 'skiing',
 'fare',
 'landscape',
 'yacht',
 'kayak',
 'beach',
 'mountains',
 'volcano',
 'economy',
 

In [ ]:
travel_words_set = travel_words_set.union(similar_words)
len(travel_words_set)

173

# Add Cities ????

In [ ]:
len(cities)

44691

In [ ]:
cities = pd.DataFrame({'Travel Keywords': cities})
df_cities = pd.concat([df, cities], ignore_index=True)

In [ ]:
df_cities.size

45178

In [ ]:
df

,Travel Keywords
0,abroad
1,adventure
2,adventurous
3,agency
4,airfare
...,...
310,sightseeing
311,skiing
312,snorkeling
313,souvenir


# Save as .csv

In [ ]:
df = pd.DataFrame({'Travel Keywords': travel_words}).assign(**{'Travel Keywords': lambda x: x['Travel Keywords'].str.lower()})

In [ ]:
df

,Travel Keywords
0,abroad
1,adventure
2,adventurous
3,agency
4,airfare
...,...
310,sightseeing
311,skiing
312,snorkeling
313,souvenir


In [ ]:
df.to_csv('298_compiled_travel_keywords.csv', index=False)

# Save as Text File

In [ ]:
pattern = '|'.join(df['Travel Keywords'])
pattern

'abroad|adventure|adventurous|agency|airfare|aircraft|airline|airport|aisle|all-inclusive|amusement park|arrival|backpack|backpacker|baggage|beach|bicycle|booking|border|breathtaking|budget|bus|cabin|campground|camping|cancellation|car rental|cruise|culture|currency|departure|destination|detour|discovery|diving|driving|economy|embark|escapade|excursion|expedition|experience|exploration|explorer|festival|ferry|foreign|frontier|getaway|globetrotter|guidebook|hiking|holiday|hostel|hotel|insurance|itinerary|jet lag|journey|journal|kayak|landmark|luggage|maps|mountains|museum|navigation|nomad|ocean|outdoor|package|passport|photography|pilgrimage|reservation|resort|restroom|road trip|route|safari|sailing|scenic|sightseeing|skiing|snorkeling|souvenir|spa|sunset|sunrise|surfing|swimming|taxi|tent|ticket|tour|tourism|tourist|touristy|trail|train|transportation|travel|traveler|trip|vacation|visa|volcano|voyage|wander|wanderer|wandering|waterfall|wildlife|yacht|zip line|zoo|adventure|aerial|agenc

In [ ]:
# Saving the string to a file
with open('travel_regex.txt', 'w') as file:
    file.write(pattern)

# # Reading the string from the file
# with open('travel_regex.txt', 'r') as file:
#     my_string = file.read()